# Loading Data

## Imports

In [1]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Flatten, LeakyReLU, ReLU, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, concatenate, Activation
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras.backend import clear_session
# from tensorflow.keras.optimizers import Adam, RMSprop , SGD
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# from tensorflow.keras.regularizers import L2
# from tensorflow.keras import metrics
from tensorflow.keras import layers
from tensorflow import keras
import keras_tuner as kt
from keras import backend as K

import tensorflow as tf

from os.path import join
from os import listdir
from shutil import copy

import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cv2

## Creating data loaders

In [2]:
IMAGE_SIZE = (64,64)

In [3]:
df = pd.read_csv('.\\final_data.csv',  names=["path","value"])
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    # # rescale=1./255
        # rescale=1.,
        # width_shift_range = 0.5,
        # height_shift_range = 0.5, 
        # zoom_range = 0.2,
        # shear_range = 0.2,
        # horizontal_flip = True,
        # vertical_flip = True,
    #  channel_shift_range = 64.0,
        # brightness_range = (0.7,1.0),
        # rotation_range = 45,
    )

data_generator = generator.flow_from_dataframe(
    df, 
    x_col="path", 
    y_col="value", 
    class_mode='raw', 
    batch_size=198,
    target_size=IMAGE_SIZE,
    )

Found 196 validated image filenames.


In [4]:
Xs = []
Ys = []
iterations = 3
for i in range(iterations):
    x,y = next(data_generator)
    Xs.extend([np.array(value).astype(int) for value in x])
    Ys.extend([np.array(value.replace("'","")[1:-1].split(', ')).astype(float) for value in y])

In [5]:
MAXPH = np.max(np.array(Ys)[:,1])
MINPH = np.min(np.array(Ys)[:,1])

MAXMOISTURE = np.max(np.array(Ys)[:,0])
MINMOISTURE = np.min(np.array(Ys)[:,0])

In [6]:
def get_mask(image):
    image = image.astype(np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    binr = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    binr = np.invert(binr)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.erode(binr, kernel, iterations=3)
    
    return mask

def process_image(image):
    rgb_image = image.astype(np.uint8)

    rgb_planes = cv2.split(rgb_image)
    result_planes = []
    for plane in rgb_planes:
        processed_image = cv2.medianBlur(plane, 2)
        result_planes.append(processed_image)
    result = cv2.merge(result_planes)

    return result

In [7]:
def process_label(image, label, maxPh, minPh, maxMoisture, minMoisture):
    mask = get_mask(np.array(image))
    shape = mask.shape 
    array = np.ones(shape=(shape[0], shape[1], 1))

    moisture = cv2.bitwise_and(array, array, mask=mask)
    ph = cv2.bitwise_and(array, array, mask=mask)

    moisture_value = (label[0] - minMoisture) / (maxMoisture - minMoisture)
    ph_value = (label[1] - minPh) / (maxPh - minPh)
    moisture[moisture > 0] = moisture_value
    ph[ph > 0] = ph_value

    output = np.stack([moisture, ph], axis=-1)  

    return output

# def process_label(image, label, maxPh, minPh, maxMoisture, minMoisture):
#     # mask = get_mask(np.array(image))
#     # shape = mask.shape 
#     # array = np.ones(shape=(shape[0], shape[1], 1))

#     # moisture = cv2.bitwise_and(array, array, mask=mask)
#     # ph = cv2.bitwise_and(array, array, mask=mask)

#     moisture_value = (label[0] - minMoisture) / (maxMoisture - minMoisture)
#     ph_value = (label[1] - minPh) / (maxPh - minPh)
#     # moisture[moisture > 0] = moisture_value
#     # ph[ph > 0] = ph_value

#     output = np.stack([moisture_value, ph_value], axis=-1)  

#     return output



In [100]:
Y_values = []
X_values = []
for i, x in enumerate(Xs):
    print(i)
    Y_values.append(process_label(Xs[i], Ys[i], MAXPH, MINPH, MAXMOISTURE, MINMOISTURE))
    # Y_values.append(process_label(Ys[i], MAXPH, MINPH, MAXMOISTURE, MINMOISTURE))
    X_values.append(process_image(Xs[i]))

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_values,Y_values , 
                                #    random_state=23,  
                                   test_size=0.20,  
                                   shuffle=True) 

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
        width_shift_range = 0.1,
        height_shift_range = 0.1, 
        # zoom_range = 0.2,
        # shear_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
    #  channel_shift_range = 64.0,
        brightness_range = (0.8,1.0),
        rotation_range = 15,
    ).flow(x=np.array(X_train), y=y_train, batch_size=6) 

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
    ).flow(x=np.array(X_test), y=y_test, batch_size=6) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [101]:
for i in next(train_generator)[1][0]:
    print(i)

[[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.58974359 0.14814815]
 [0.58974359 0.14814815]
 [0.58974359 0.14814815]
 [0.58974359 0.14814815]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]


In [102]:
x_col = "path"
y_col = "pH"
batch_size = 16
epochs = 1024
lr = 1e-5
image_size = (IMAGE_SIZE[0],IMAGE_SIZE[1])
channels = 3
shuffle = True
class_mode ="raw"
color_mode = "rgb"

In [103]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      width_shift_range = 0.2,
#                                      height_shift_range = 0.2, 
#                                      zoom_range = 0.2,
#                                      shear_range = 0.2,
#                                      horizontal_flip = True,
#                                      vertical_flip = True,
#                                     #  channel_shift_range = 64.0,
#                                      brightness_range = (0.5,1.0),
#                                      rotation_range = 45,
#                                      )
# train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
#                                               x_col=x_col, y_col=y_col, has_ext=True, 
#                                               class_mode=class_mode, target_size=image_size, 
#                                               batch_size=batch_size, color_mode = color_mode,
#                                               shuffle = shuffle)
# val_datagen = ImageDataGenerator(rescale=1./255)
# val_generator = val_datagen.flow_from_dataframe(dataframe=val_df,
#                                               x_col=x_col, y_col=y_col, has_ext=True, 
#                                               class_mode=class_mode, target_size=image_size, 
#                                               batch_size=batch_size,  color_mode = color_mode,
#                                               shuffle = shuffle)


# Design Alternatives

## Design A: Nested U-Net

In [178]:
# Defining the Convolutional Block
def conv_block(inputs, num_filters):
	# Applying the sequence of Convolutional, Batch Normalization
	# and Activation Layers to the input tensor
	x = Sequential([
		# Convolutional Layer
		Conv2D(num_filters, 1, padding='same'),
		# Batch Normalization Layer
		BatchNormalization(),
		# Activation Layer
		ReLU(),
		# Convolutional Layer
		Conv2D(num_filters, 1, padding='same'),
		# Batch Normalization Layer
		BatchNormalization(),
		# Activation Layer
		ReLU()
	])(inputs)

	# Returning the output of the Convolutional Block
	return x

# Defining the Unet++ Model
def unet_plus_plus_model(input_shape=(image_size[0], image_size[1], channels), num_classes=1, deep_supervision=True):
	inputs = Input(shape=input_shape)

	# Encoding Path
	x_00 = conv_block(inputs, 64)
	x_10 = conv_block(MaxPooling2D()(x_00), 128)
	x_20 = conv_block(MaxPooling2D()(x_10), 256)
	x_30 = conv_block(MaxPooling2D()(x_20), 512)
	x_40 = conv_block(MaxPooling2D()(x_30), 1024)

	# Nested Decoding Path
	x_01 = conv_block(concatenate(
		[x_00, UpSampling2D()(x_10)]), 64)
	x_11 = conv_block(concatenate(
		[x_10, UpSampling2D()(x_20)]), 128)
	x_21 = conv_block(concatenate(
		[x_20, UpSampling2D()(x_30)]), 256)
	x_31 = conv_block(concatenate(
		[x_30, UpSampling2D()(x_40)]), 512)

	x_02 = conv_block(concatenate(
		[x_00, x_01, UpSampling2D()(x_11)]), 64)
	x_12 = conv_block(concatenate(
		[x_10, x_11, UpSampling2D()(x_21)]), 128)
	x_22 = conv_block(concatenate(
		[x_20, x_21, UpSampling2D()(x_31)]), 256)

	x_03 = conv_block(concatenate(
		[x_00, x_01, x_02, UpSampling2D()(x_12)]), 64)
	x_13 = conv_block(concatenate(
		[x_10, x_11, x_12, UpSampling2D()(x_22)]), 128)

	x_04 = conv_block(concatenate(
		[x_00, x_01, x_02, x_03, UpSampling2D()(x_13)]), 64)

	# Deep Supervision Path
	# If deep supervision is enabled, then the model will output the segmentation maps
	# at each stage of the decoding path
	if deep_supervision:
		outputs = [
			Conv2D(num_classes, 1)(x_01),
			Conv2D(num_classes, 1)(x_02),
			Conv2D(num_classes, 1)(x_03),
			Conv2D(num_classes, 1)(x_04)
		]
		# Concatenating the segmentation maps
		outputs = concatenate(outputs, axis=0)

	# If deep supervision is disabled, then the model will output the final segmentation map
	# which is the segmentation map at the end of the decoding path
	else:
		# flatten  = layers.Flatten()(x_04)
		# dense = Dense(1024, activation='relu')(flatten)
		# dense = Dense(512, activation='relu')(dense)
		# outputs = Dense(2, activation='sigmoid')(flatten)
		outputs = tf.keras.layers.Conv2D(2, 1, activation='sigmoid')(x_04)
	# Creating the model
	model = tf.keras.Model(
		inputs=inputs, outputs=outputs, name='Unet_plus_plus')

	# Returning the model
	return model




Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

c:\Users\nealb\AppData\Local\miniconda3\envs\pd\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 2, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_block2_out (Activation)  (None, 2, 2, 2048)   0           ['conv5_block2_add[0][0]']       
                                                                                                  
 conv5_block3_1_conv (Conv2D)   (None, 2, 2, 512)    1049088     ['conv5_block2_out[0][0]']       
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, 2, 2, 512)   2048        ['conv5_block3_1_conv[0][0]']    
 ization) 

In [46]:
# Creating the model
modelA = unet_plus_plus_model(input_shape=(
	image_size[0], image_size[1], channels), deep_supervision=False)
modelA.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy', metrics=['acc'])
modelA.summary()

Model: "Unet_plus_plus"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 sequential_18 (Sequential)     (None, 32, 32, 64)   4928        ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 16, 16, 64)  0           ['sequential_18[0][0]']          
                                                                                                  
 sequential_19 (Sequential)     (None, 16, 16, 128)  25856       ['max_pooling2d_4[0][0]']        
                                                                                     

In [ ]:


# historyA = modelA.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
# ## Graphing Result of training
# plt.figure(figsize=(15,4))

# plt.subplot(121)
# plt.plot(historyA.history['mae'], color ='r')
# plt.plot(historyA.history['val_mae'])
# plt.title('mae')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mae')

# plt.subplot(122)
# plt.plot(historyA.history['loss'], color ='r')
# plt.plot(historyA.history['val_loss'])
# plt.title('loss')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mean squared error')

# # displaying the figure
# plt.show()

In [ ]:
# ## Best model saved
# train_loss = historyA.history['loss']
# train_mae = historyA.history['mae']
# val_loss = historyA.history['val_loss']
# val_mae = historyA.history['val_mae']

# best_epoch = np.argmin(np.array(val_loss))

# print("Best Epoch: ", best_epoch)
# print("Final Train Loss: {:.4f}".format(train_loss[best_epoch]))
# print("Final Validation Loss: {:.4f}".format(val_loss[best_epoch]))
# print("Final Train MAE: {:.2f}".format(train_mae[best_epoch]*100))
# print("Final Test MAE: {:.2f}".format(val_mae[best_epoch]))

## Design B: SegNet

In [ ]:
def segnet(input_shape):

    # Encoding layer
    img_input = Input(shape= input_shape)
    x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(img_input)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
    x = BatchNormalization(name='bn3')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
    x = BatchNormalization(name='bn5')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
    x = BatchNormalization(name='bn6')(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
    x = BatchNormalization(name='bn7')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
    x = BatchNormalization(name='bn8')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
    x = BatchNormalization(name='bn9')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
    x = BatchNormalization(name='bn10')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
    x = BatchNormalization(name='bn11')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
    x = BatchNormalization(name='bn12')(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
    x = BatchNormalization(name='bn13')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Dense(1024, activation = 'relu', name='fc1')(x)
    x = Dense(1024, activation = 'relu', name='fc2')(x)
    # Decoding Layer 
    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
    x = BatchNormalization(name='bn14')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
    x = BatchNormalization(name='bn15')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
    x = BatchNormalization(name='bn16')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
    x = BatchNormalization(name='bn17')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
    x = BatchNormalization(name='bn18')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
    x = BatchNormalization(name='bn19')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
    x = BatchNormalization(name='bn20')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
    x = BatchNormalization(name='bn21')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
    x = BatchNormalization(name='bn22')(x)
    x = Activation('relu')(x)

    x = UpSampling2D()(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
    x = BatchNormalization(name='bn23')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
    x = BatchNormalization(name='bn24')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
    x = BatchNormalization(name='bn25')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
    x = BatchNormalization(name='bn26')(x)
    x = Activation('sigmoid')(x)
    # pred = Reshape((192,256))(x)
    # x = Flatten()(x)
    # x=Dense(1024, activation='relu')(x)
    # otuput=Dense(4, activation='softmax')(x)
    outputs = tf.keras.layers.Conv2D(2, 1, activation='sigmoid')(x)
    
    model = Model(inputs=img_input, outputs=outputs)
    
    model.compile(optimizer= tf.keras.optimizers.Adam(lr=lr), loss= ["binary_crossentropy"]
                  , metrics=['acc'])
    return model

In [ ]:

modelB = segnet(input_shape=image_size+(3,))
modelB.summary()


# model.save(savename)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 32, 32, 64)        1792      
                                                                 
 bn1 (BatchNormalization)    (None, 32, 32, 64)        256       
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 conv2 (Conv2D)              (None, 32, 32, 64)        36928     
                                                                 
 bn2 (BatchNormalization)    (None, 32, 32, 64)        256       
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0     

c:\Users\nealb\AppData\Local\miniconda3\envs\pd\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# hist = modelB.fit(train_generator, epochs= 10, validation_data= val_generator, verbose=1)

In [19]:
# modelB = segnet(input_shape=image_size+(3,))
# modelB.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='mse', metrics=['mae'])
# modelB.summary()

In [20]:
# ## MANUFACTURABILITY: TRAINING TIME
# import time 
# start = time.time()
# historyB = modelB.fit(train_generator, epochs=10, validation_data=val_generator, verbose=0)
# stop = time.time()
# print(f"Training time: {stop - start}s")



In [21]:
# ## Graphing Result of training
# plt.figure(figsize=(15,4))

# plt.subplot(121)
# plt.plot(historyB.history['mae'], color ='r')
# plt.plot(historyB.history['val_mae'])
# plt.title('mae')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mae')

# plt.subplot(122)
# plt.plot(historyB.history['loss'], color ='r')
# plt.plot(historyB.history['val_loss'])
# plt.title('loss')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mean squared error')

# # displaying the figure
# plt.show()

In [22]:
# ## Best model saved
# train_loss = historyB.history['loss']
# train_mae = historyB.history['mae']
# val_loss = historyB.history['val_loss']
# val_mae = historyB.history['val_mae']

# best_epoch = np.argmin(np.array(val_loss))

# print("Best Epoch: ", best_epoch)
# print("Final Train Loss: {:.4f}".format(train_loss[best_epoch]))
# print("Final Validation Loss: {:.4f}".format(val_loss[best_epoch]))
# print("Final Train MAE: {:.2f}".format(train_mae[best_epoch]*100))
# print("Final Test MAE: {:.2f}".format(val_mae[best_epoch]))

In [23]:
# train_generator[1][1]

## Design C: DeepLabv3+


In [24]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def DeeplabV3Plus(image_size):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights=None, include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    # x = layers.GlobalAveragePooling2D()(x)
    # x= Dense(1024, activation="relu",)(x)
    # model_output = layers.Dense(4, activation='softmax')(x)
    outputs = tf.keras.layers.Conv2D(2, 1, activation='sigmoid')(x)
    return keras.Model(inputs=model_input, outputs=outputs)


modelC = DeeplabV3Plus(image_size=image_size[0])
modelC.summary()

In [ ]:
modelC.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy', metrics=['acc'])

# historyC = modelC.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
# ## Graphing Result of training
# plt.figure(figsize=(15,4))

# plt.subplot(121)
# plt.plot(historyC.history['mae'], color ='r')
# plt.plot(historyC.history['val_mae'])
# plt.title('mae')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mae')

# plt.subplot(122)
# plt.plot(historyC.history['loss'], color ='r')
# plt.plot(historyC.history['val_loss'])
# plt.title('loss')
# plt.legend(['train','validation'])
# plt.xlabel('epoch')
# plt.ylabel('mean squared error')

# # displaying the figure
# plt.show()

In [ ]:
# ## Best model saved
# train_loss = historyC.history['loss']
# train_mae = historyC.history['mae']
# val_loss = historyC.history['val_loss']
# val_mae = historyC.history['val_mae']

# best_epoch = np.argmin(np.array(val_loss))

# print("Best Epoch: ", best_epoch)
# print("Final Train Loss: {:.4f}".format(train_loss[best_epoch]))
# print("Final Validation Loss: {:.4f}".format(val_loss[best_epoch]))
# print("Final Train MAE: {:.2f}".format(train_mae[best_epoch]*100))
# print("Final Test MAE: {:.2f}".format(val_mae[best_epoch]))

# Evaluation of Designs

## Optimization of Designs

### Design A

In [184]:
# Defining the Convolutional Block
def conv_block(inputs, num_filters):
	# Applying the sequence of Convolutional, Batch Normalization
	# and Activation Layers to the input tensor
	x = Sequential([
		# Convolutional Layer
		Conv2D(num_filters, 1, padding='same'),
		# Batch Normalization Layer
		BatchNormalization(),
		# Activation Layer
		ReLU(),
		# Convolutional Layer
		Conv2D(num_filters, 1, padding='same'),
		# Batch Normalization Layer
		BatchNormalization(),
		# Activation Layer
		ReLU()
	])(inputs)

	# Returning the output of the Convolutional Block
	return x
def dense_block(units, dropout_rate):
    return Sequential([
        Dense(units, activation='relu'),
    ])
# Defining the Unet++ Model
def unet_plus_plus_model(hp):
	inputs = Input(shape=image_size+(3,))
	hp_filters = hp.Choice('filters',values = [16,32,64])
	# Encoding Path
	x_00 = conv_block(inputs, hp_filters)
	x_10 = conv_block(MaxPooling2D()(x_00), hp_filters*2)
	x_20 = conv_block(MaxPooling2D()(x_10), hp_filters*4)
	x_30 = conv_block(MaxPooling2D()(x_20), hp_filters*8)
	x_40 = conv_block(MaxPooling2D()(x_30), hp_filters*16)
	
	hp.Boolean("dropouts", default=False)
	hp.Boolean("batch_normalization", default=False)
	flattened = Flatten()(x_40)
	dense = dense_block(hp_filters*hp_filters, 0.2)(flattened)
	if hp.Boolean("dropouts"):
		dense = tf.keras.layers.Dropout(0.5)(dense)
	if hp.Boolean("batch_normalization"):
		dense = tf.keras.layers.BatchNormalization()(dense)
	dense = dense_block(hp_filters*hp_filters, 0.2)(dense)
	if hp.Boolean("dropouts"):
		dense = tf.keras.layers.Dropout(0.5)(dense)
	if hp.Boolean("batch_normalization"):
		dense = tf.keras.layers.BatchNormalization()(dense)
	dense = dense_block(hp_filters*hp_filters, 0.2)(dense);hp.Boolean("4th dense", default=False);
	if hp.Boolean("4th dense"):
		dense = dense_block(hp_filters*hp_filters, 0.2)(dense)
		if hp.Boolean("dropouts"):
			dense = tf.keras.layers.Dropout(0.5)(dense)
		if hp.Boolean("batch_normalization"):
			dense = tf.keras.layers.BatchNormalization()(dense)
	# dense = dense_block(4096, 0.2)(dense)
	# dense = dense_block(4096, 0.2)(dense)
	reshaped = tf.keras.layers.Reshape((x_40.shape[1], x_40.shape[1], (hp_filters*hp_filters)//(x_40.shape[1]*x_40.shape[1])))(dense)  # Reshape to reintroduce spatial dimensions


	# Nested Decoding Path
	x_01 = conv_block(concatenate(
		[x_00, UpSampling2D()(x_10)]), hp_filters)
	x_11 = conv_block(concatenate(
		[x_10, UpSampling2D()(x_20)]), hp_filters*2)
	x_21 = conv_block(concatenate(
		[x_20, UpSampling2D()(x_30)]), hp_filters*4)
	x_31 = conv_block(concatenate(
		[x_30, UpSampling2D()(reshaped)]), hp_filters*8)

	x_02 = conv_block(concatenate(
		[x_00, x_01, UpSampling2D()(x_11)]), hp_filters)
	x_12 = conv_block(concatenate(
		[x_10, x_11, UpSampling2D()(x_21)]), hp_filters*2)
	x_22 = conv_block(concatenate(
		[x_20, x_21, UpSampling2D()(x_31)]), hp_filters*4)

	x_03 = conv_block(concatenate(
		[x_00, x_01, x_02, UpSampling2D()(x_12)]), hp_filters)
	x_13 = conv_block(concatenate(
		[x_10, x_11, x_12, UpSampling2D()(x_22)]), hp_filters*2)

	x_04 = conv_block(concatenate(
		[x_00, x_01, x_02, x_03, UpSampling2D()(x_13)]), hp_filters)
	
	outputs = tf.keras.layers.Conv2D(2, 1, activation='sigmoid')(x_04);print(outputs.shape)

	# Creating the model
	model = tf.keras.Model(
		inputs=inputs, outputs=outputs, name='Unet_plus_plus');lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5]);model.compile(optimizer= tf.keras.optimizers.Adam(lr=lr), loss= ["binary_crossentropy"], metrics=['acc'])
	# Returning the model
	return model




In [247]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Dropout, Concatenate, Flatten, Dense, Reshape
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.initializers import HeNormal

def convolution_block(inputs, num_filters, kernel_size=3, padding="same", use_bias=False):
    x = Conv2D(num_filters, kernel_size=kernel_size, padding=padding, use_bias=use_bias, kernel_initializer=HeNormal())(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def upsample_block(inputs, skip_features, num_filters):
    x = UpSampling2D((2, 2))(inputs)
    x = Conv2D(num_filters, (2, 2), padding="same")(x)
    x = Concatenate()([x, skip_features])
    x = convolution_block(x, num_filters)
    return x

def dense_upsampling_block(inputs, skip_features, num_filters):
    x = UpSampling2D((2, 2))(inputs)
    x = Conv2D(num_filters, (2, 2), padding="same")(x)
    x = Concatenate()([x, skip_features])
    x = convolution_block(x, num_filters)
    return x

def unet_plus_plus_model(hp,input_shape=(64,64,3)):
    inputs = Input(input_shape)

    resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    resnet50.trainable = True

    # Encoder
    s1 = resnet50.get_layer("conv1_relu").output
    s2 = resnet50.get_layer("conv2_block3_out").output
    s3 = resnet50.get_layer("conv3_block4_out").output
    s4 = resnet50.get_layer("conv4_block6_out").output

    b1 = resnet50.get_layer("conv5_block3_out").output
    # Flatten the bottleneck output
    x = Flatten()(b1)
	UNITS = hp.Choice('units',values = [256,512,1024,2048])
    
    # Dense layers between encoder and decoder
    x = Dense(UNITS, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(UNITS, activation='relu')(x)
    x = Dense(UNITS, activation='relu')(x)
    # Reshape back to spatial dimensions for the decoder
    SIZE = input_shape[0] // 32  # Assuming input size is a multiple of 32
    x = Dense(SIZE * SIZE * 2048, activation='relu')(x)
    x = Reshape((SIZE, SIZE, 2048))(x)

    # Nested U-Net
    d4_2 = dense_upsampling_block(x, s4, 512)
    d3_2 = dense_upsampling_block(d4_2, s3, 256)
    d2_2 = dense_upsampling_block(d3_2, s2, 128)
    d1_2 = dense_upsampling_block(d2_2, s1, 64)

    d4_1 = upsample_block(x, s4, 512)
    d3_1 = upsample_block(d4_1, s3, 256)
    d2_1 = upsample_block(d3_1, s2, 128)
    d1_1 = upsample_block(d2_1, s1, 64)

    outputs = UpSampling2D()(d1_1)
    outputs = Conv2D(2, (1, 1), padding="same", activation="sigmoid")(outputs)
    model = tf.keras.Model(
		inputs=inputs, outputs=outputs, name='Unet_plus_plus');lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5]);model.compile(optimizer= tf.keras.optimizers.Adam(lr=lr), loss= ["binary_crossentropy"], metrics=['acc'])
	# Returning the model
    return model

# # Define input shape and build model
# input_shape = (256, 256, 3)
# model = build_resnet50_unetpp(input_shape)
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

# # Print model summary
# model.summary()


In [248]:
# # Define the convolutional block
# def conv_block(inputs, num_filters):
#     x = Sequential([
#         Conv2D(num_filters, 3, padding='same'),
#         BatchNormalization(),
#         ReLU(),
#         Conv2D(num_filters, 3, padding='same'),
#         BatchNormalization(),
#         ReLU()
#     ])(inputs)
#     return x

# # Define the dense block
# def dense_block(units, dropout_rate):
#     return Sequential([
#         Dense(units, activation='relu'),
#         Dropout(dropout_rate)
#     ])

# # Define the Unet++ model
# def unet_plus_plus_model(hp, image_size=(224, 224)):
#     inputs = Input(shape=image_size + (3,))

#     # Load ResNet50 as encoder
#     resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
#     encoder_layers = [resnet.get_layer(name).output for name in ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']]
    
#     # Encoding path
#     x_00 = encoder_layers[0]
#     x_10 = encoder_layers[1]
#     x_20 = encoder_layers[2]
#     x_30 = encoder_layers[3]
#     x_40 = encoder_layers[4]

#     flattened = Flatten()(x_40)
#     dense = dense_block(2048, 0.2)(flattened)
#     if hp.Boolean("dropouts"):
#         dense = Dropout(0.5)(dense)
#     if hp.Boolean("batch_normalization"):
#         dense = BatchNormalization()(dense)
#     dense = dense_block(2048, 0.2)(dense)
#     if hp.Boolean("dropouts"):
#         dense = Dropout(0.5)(dense)
#     if hp.Boolean("batch_normalization"):
#         dense = BatchNormalization()(dense)
#     dense = dense_block(2048, 0.2)(dense)
#     if hp.Boolean("4th_dense"):
#         dense = dense_block(2048, 0.2)(dense)
#         if hp.Boolean("dropouts"):
#             dense = Dropout(0.5)(dense)
#         if hp.Boolean("batch_normalization"):
#             dense = BatchNormalization()(dense)

#     reshaped = Reshape((x_40.shape[1], x_40.shape[2], 32))(dense)

#     # Nested decoding path
#     x_01 = conv_block(concatenate([x_00, UpSampling2D()(x_10)]), 64)
#     x_11 = conv_block(concatenate([x_10, UpSampling2D()(x_20)]), 128)
#     x_21 = conv_block(concatenate([x_20, UpSampling2D()(x_30)]), 256)
#     x_31 = conv_block(concatenate([x_30, UpSampling2D()(reshaped)]), 512)

#     x_02 = conv_block(concatenate([x_00, x_01, UpSampling2D()(x_11)]), 64)
#     x_12 = conv_block(concatenate([x_10, x_11, UpSampling2D()(x_21)]), 128)
#     x_22 = conv_block(concatenate([x_20, x_21, UpSampling2D()(x_31)]), 256)

#     x_03 = conv_block(concatenate([x_00, x_01, x_02, UpSampling2D()(x_12)]), 64)
#     x_13 = conv_block(concatenate([x_10, x_11, x_12, UpSampling2D()(x_22)]), 128)

#     x_04 = conv_block(concatenate([x_00, x_01, x_02, x_03, UpSampling2D()(x_13)]), 64)

#     outputs = Conv2D(2, 1, activation='sigmoid')(x_04)

#     # Create the model
#     model = Model(inputs=inputs, outputs=outputs, name='Unet_plus_plus')
#     lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="binary_crossentropy", metrics=['acc'])

#     return model

In [249]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, ReLU, Flatten, Dense, Dropout, concatenate, Reshape
# import tensorflow as tf
# import keras_tuner as kt

# # Define the convolutional block
# def conv_block(inputs, num_filters):
#     x = Sequential([
#         Conv2D(num_filters, 3, padding='same'),
#         BatchNormalization(),
#         ReLU(),
#         Conv2D(num_filters, 3, padding='same'),
#         BatchNormalization(),
#         ReLU()
#     ])(inputs)
#     return x

# # Define the dense block
# def dense_block(units, dropout_rate):
#     return Sequential([
#         Dense(units, activation='relu'),
#         Dropout(dropout_rate)
#     ])

# # Define the Unet++ model
# def unet_plus_plus_model(hp, image_size=(32, 32)):
#     inputs = Input(shape=image_size + (3,))

#     # Load ResNet50 as encoder
#     resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
#     encoder_layers = [resnet.get_layer(name).output for name in ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']]
    
#     # Encoding path
#     x_00 = encoder_layers[0]
#     x_10 = encoder_layers[1]
#     x_20 = encoder_layers[2]
#     x_30 = encoder_layers[3]
#     x_40 = encoder_layers[4]
#     print(x_40.shape)
#     flattened = Flatten()(x_40)
#     dense = dense_block(2048, 0.2)(flattened)
#     if hp.Boolean("dropouts"):
#         dense = Dropout(0.5)(dense)
#     if hp.Boolean("batch_normalization"):
#         dense = BatchNormalization()(dense)
#     dense = dense_block(2048, 0.2)(dense)
#     if hp.Boolean("dropouts"):
#         dense = Dropout(0.5)(dense)
#     if hp.Boolean("batch_normalization"):
#         dense = BatchNormalization()(dense)
#     dense = dense_block(2048, 0.2)(dense)
#     if hp.Boolean("4th_dense"):
#         dense = dense_block(2048, 0.2)(dense)
#         if hp.Boolean("dropouts"):
#             dense = Dropout(0.5)(dense)
#         if hp.Boolean("batch_normalization"):
#             dense = BatchNormalization()(dense)
#     reshaped = tf.keras.layers.Reshape((x_40.shape[1], x_40.shape[1], 2048))(dense)  # Reshape to reintroduce spatial dimensions

#     # Nested decoding path
#     x_01 = conv_block(concatenate([x_00, UpSampling2D()(x_10)]), 64)
#     x_11 = conv_block(concatenate([x_10, UpSampling2D()(x_20)]), 128)
#     x_21 = conv_block(concatenate([x_20, UpSampling2D()(x_30)]), 256)
#     x_31 = conv_block(concatenate([x_30, UpSampling2D()(reshaped)]), 512)

#     x_02 = conv_block(concatenate([x_00, x_01, UpSampling2D()(x_11)]), 64)
#     x_12 = conv_block(concatenate([x_10, x_11, UpSampling2D()(x_21)]), 128)
#     x_22 = conv_block(concatenate([x_20, x_21, UpSampling2D()(x_31)]), 256)

#     x_03 = conv_block(concatenate([x_00, x_01, x_02, UpSampling2D()(x_12)]), 64)
#     x_13 = conv_block(concatenate([x_10, x_11, x_12, UpSampling2D()(x_22)]), 128)

#     x_04 = conv_block(concatenate([x_00, x_01, x_02, x_03, UpSampling2D()(x_13)]), 64)
    
#     x_final = UpSampling2D(size=(2, 2))(x_04)

#     outputs = Conv2D(2, 1, activation='sigmoid')(x_final)
#     print(outputs.shape)

#     # Create the model
#     model = Model(inputs=inputs, outputs=outputs, name='Unet_plus_plus')
#     lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="binary_crossentropy", metrics=['acc'])

#     return model



In [250]:
# # Define the convolutional block
# def conv_block(inputs, num_filters):
#     x = Sequential([
#         Conv2D(num_filters, 1, padding='same'),
#         BatchNormalization(),
#         ReLU(),
#         Conv2D(num_filters, 1, padding='same'),
#         BatchNormalization(),
#         ReLU()
#     ])(inputs)
#     return x

# # Define the dense block
# def dense_block(units, dropout_rate):
#     return Sequential([
#         Dense(units, activation='relu')
#     ])

# # Define the Unet++ model
# def unet_plus_plus_model(hp):
#     inputs = Input(shape=(32, 32, 3))
#     hp_filters = hp.Choice('filters', values=[16, 32, 64,])

#     # Encoding Path
#     x_00 = conv_block(inputs, hp_filters)
#     x_10 = conv_block(MaxPooling2D()(x_00), hp_filters * 2)
#     x_20 = conv_block(MaxPooling2D()(x_10), hp_filters * 4)
#     x_30 = conv_block(MaxPooling2D()(x_20), hp_filters * 8)
#     x_40 = conv_block(MaxPooling2D()(x_30), hp_filters * 16)

#     # Dense layers and reshape
#     flattened = Flatten()(x_40)
#     dense = dense_block(hp_filters * hp_filters, 0.2)(flattened)
#     if hp.Boolean("dropouts"):
#         dense = Dropout(0.5)(dense)
#     if hp.Boolean("batch_normalization"):
#         dense = BatchNormalization()(dense)
#     dense = dense_block(hp_filters * hp_filters, 0.2)(dense)
#     if hp.Boolean("dropouts"):
#         dense = Dropout(0.5)(dense)
#     if hp.Boolean("batch_normalization"):
#         dense = BatchNormalization()(dense)
#     dense = dense_block(hp_filters * hp_filters, 0.2)(dense)
#     if hp.Boolean("4th_dense"):
#         dense = dense_block(hp_filters * hp_filters, 0.2)(dense)
#         if hp.Boolean("dropouts"):
#             dense = Dropout(0.5)(dense)
#         if hp.Boolean("batch_normalization"):
#             dense = BatchNormalization()(dense)

#     reshaped = tf.keras.layers.Reshape((x_40.shape[1], x_40.shape[1], (hp_filters * hp_filters) // (x_40.shape[1] * x_40.shape[1])))(dense)

#     # Nested Decoding Path
#     x_01 = conv_block(concatenate([x_00, UpSampling2D()(x_10)]), hp_filters)
#     x_11 = conv_block(concatenate([x_10, UpSampling2D()(x_20)]), hp_filters * 2)
#     x_21 = conv_block(concatenate([x_20, UpSampling2D()(x_30)]), hp_filters * 4)
#     x_31 = conv_block(concatenate([x_30, UpSampling2D()(reshaped)]), hp_filters * 8)

#     x_02 = conv_block(concatenate([x_00, x_01, UpSampling2D()(x_11)]), hp_filters)
#     x_12 = conv_block(concatenate([x_10, x_11, UpSampling2D()(x_21)]), hp_filters * 2)
#     x_22 = conv_block(concatenate([x_20, x_21, UpSampling2D()(x_31)]), hp_filters * 4)

#     x_03 = conv_block(concatenate([x_00, x_01, x_02, UpSampling2D()(x_12)]), hp_filters)
#     x_13 = conv_block(concatenate([x_10, x_11, x_12, UpSampling2D()(x_22)]), hp_filters * 2)

#     x_04 = conv_block(concatenate([x_00, x_01, x_02, x_03, UpSampling2D()(x_13)]), hp_filters)

#     # Flattening the final output
#     flat_output = Flatten()(x_04)

#     # Adding Dense layers
#     dense_output = Dense(hp_filters * hp_filters, activation='relu')(flat_output)
#     if hp.Boolean("dropouts"):
#         dense_output = Dropout(0.5)(dense_output)
#     if hp.Boolean("batch_normalization"):
#         dense_output = BatchNormalization()(dense_output)

#     # Final Dense layer with sigmoid activation for binary classification
#     outputs = Dense(2, activation='sigmoid')(dense_output)

#     # Create the model
#     model = Model(inputs=inputs, outputs=outputs, name='Unet_plus_plus')
#     lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="binary_crossentropy", metrics=['acc'])

#     return model

In [251]:
tunerA = kt.BayesianOptimization(unet_plus_plus_model,
                     objective='val_loss',
                     directory='my_dir',
                     max_trials= 10,
                     project_name='design_a',
                    #  seed=42,
                     )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

tunerA.search(train_generator, epochs=100, validation_data=val_generator, callbacks=[stop_early, reduce_lr])

# Get the optimal hyperparameters
best_hpsA=tunerA.get_best_hyperparameters(num_trials=1)[0]


print(best_hpsA.values)

Trial 3 Complete [00h 05m 44s]
val_loss: 0.31658250093460083

Best val_loss So Far: 0.31658250093460083
Total elapsed time: 00h 12m 37s
{'learning_rate': 0.0001}


In [17]:
# best_hpsA.values['filters']=64

In [18]:

# Build the model with the best hp.
modelA = unet_plus_plus_model(best_hpsA)
modelA.summary()

(None, 64, 64, 2)
Model: "Unet_plus_plus"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 sequential_18 (Sequential)     (None, 64, 64, 32)   1440        ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 32, 32, 32)  0           ['sequential_18[0][0]']          
                                                                                                  
 sequential_19 (Sequential)     (None, 32, 32, 64)   6784        ['max_pooling2d_4[0][0]']        
                                                                   

In [19]:
# '''
#    Manual Training with y data alteration
# '''
# ## Variable declaration
# epochs = 512
# steps_per_epoch = 512 // batch_size + 1
# min_lr = 1e-8
# factor = 0.1
# SCHEDULE_EPOCH = 50
# STEPS = 50
# MONITOR = "loss"
# PATIENCE = 10
# WEIGHTS = []

# historyA = {"history": {"loss": [], "mae": [], "val_loss": [], "val_mae": []}}
# for e in range(epochs):
#     for i, (images, y_batch) in enumerate(train_generator):
#         new_y_batch = []

#         ## Train data y alteration
#         for x, img in enumerate(images):
#             array = np.ones(image_size + (3,))
#             array *= img > 0
#             array[array > 0] = y_batch[x]
#             new_y_batch.append(array)
#         new_y_batch = np.array(new_y_batch)
#         loss = modelA.train_on_batch(images, new_y_batch)  # train model for a single iteration

#         if i >= steps_per_epoch:  # manually detect the end of the epoch train set
#             historyA["history"]["loss"].append(loss[0])
#             historyA["history"]["mae"].append(loss[1])
#             break

#     for i, (images, y_batch) in enumerate(val_generator):
#         new_y_batch = []

#         ## Val data y alteration
#         for x, img in enumerate(images):
#             array = np.ones(image_size + (3,))
#             array *= img > 0
#             array[array > 0] = y_batch[x]
#             new_y_batch.append(array)
#         new_y_batch = np.array(new_y_batch)
#         val = modelA.test_on_batch(images, new_y_batch)

#         if i >= steps_per_epoch:  # manually detect the end of the epoch validation set
#             historyA["history"]["val_loss"].append(val[0])
#             historyA["history"]["val_mae"].append(val[1])
#             break
        
#     ## LR Scheduler
#     curr_lr = modelA.optimizer.learning_rate
#     if e >= SCHEDULE_EPOCH and curr_lr > min_lr:
#         K.set_value(modelA.optimizer.learning_rate, curr_lr * factor)
#         curr_lr = modelA.optimizer.learning_rate
#         SCHEDULE_EPOCH += STEPS

#     # ## Early Stopping
#     # if e > PATIENCE:
#     #     PAST = historyA["history"][MONITOR][-PATIENCE-1:-1]
#     #     LATEST = historyA["history"][MONITOR][-1]
#     #     for i,P in enumerate(PAST):
#     #         if LATEST > P:
#     #             PATIENCE -= 1  ## decrement patience if previous monitor is worse than the current value

#     # if PATIENCE <= 0:
#     #     break  ## if no more patience left, early stop training
#     # PATIENCE = 10


#     print("EPOCH: {} LOSS: {:.6f} MAE: {:.6f} VAL_LOSS: {:.6f} VAL_MAE: {:.6f}   CURR_LR {:.2E}".format(
#         e + 1, loss[0], loss[1], val[0], val[1], curr_lr.numpy()))
#     train_generator.on_epoch_end()  # shuffles the data at the end of each epoch


In [20]:
# plt.figure(figsize=(10,4))

# plt.subplot(121)
# plt.plot(historyA["history"]["loss"], color='g',alpha=.5)
# plt.plot(historyA["history"]["val_loss"], color='r',alpha=.7)
# plt.legend(["Train","Val"])
# plt.ylabel("MSE")
# plt.xlabel("Epochs")

# plt.subplot(122)
# plt.plot(historyA["history"]["mae"], color='g',alpha=.5)
# plt.plot(historyA["history"]["val_mae"], color='r',alpha=.7)
# plt.legend(["Train","Val"])
# plt.ylabel("MAE")
# plt.xlabel("Epochs")

# plt.suptitle("Model A: Nested U-Net")
# plt.show()

In [21]:
# sample = next(train_generator)


In [22]:
# print(modelA(sample[0]))
# print(sample[1])

In [23]:
# # Validation
# val_sample = next(val_generator)
# val_result = modelA.predict(val_sample[0])

# for x, i in enumerate(val_result):
#     RGB = cv2.cvtColor(val_sample[0][x], cv2.COLOR_HSV2RGB)
#     MASK = cv2.cvtColor(RGB, cv2.COLOR_RGB2GRAY) > 0
#     ARRAY = np.array(val_result[x]).reshape(image_size) * MASK
#     OUTPUT = np.array([x for x in ARRAY.flatten() if x > 0])
#     print("Validation - Average Output:", np.average(OUTPUT), "Ground Truth:", np.average(val_sample[1][x]))


In [24]:
# modelC.save('design_models/designA.h5')

In [25]:
# modelA = tf.keras.models.load_model('design_models/designA.h5')

In [26]:
# from shutil import rmtree
# # removing directory 
# rmtree('my_dir') 

### Design B

In [45]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Conv2DTranspose, Dropout, Flatten, Dense, Reshape
from tensorflow.keras import backend as K
import tensorflow as tf
import keras_tuner as kt

def segnet(hp):
    hp_filters = hp.Choice('filters', values=[16, 32, 64])
    
    # Encoding path
    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    
    # Encoder
    x = Conv2D(hp_filters, (3, 3), padding='same', name='conv1')(inputs)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters, (3, 3), padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(name='pool1')(x)
    
    x = Conv2D(hp_filters * 2, (3, 3), padding='same', name='conv3')(x)
    x = BatchNormalization(name='bn3')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 2, (3, 3), padding='same', name='conv4')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(name='pool2')(x)
    
    x = Conv2D(hp_filters * 4, (3, 3), padding='same', name='conv5')(x)
    x = BatchNormalization(name='bn5')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 4, (3, 3), padding='same', name='conv6')(x)
    x = BatchNormalization(name='bn6')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 4, (3, 3), padding='same', name='conv7')(x)
    x = BatchNormalization(name='bn7')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(name='pool3')(x)
    
    x = Conv2D(hp_filters * 8, (3, 3), padding='same', name='conv8')(x)
    x = BatchNormalization(name='bn8')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 8, (3, 3), padding='same', name='conv9')(x)
    x = BatchNormalization(name='bn9')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 8, (3, 3), padding='same', name='conv10')(x)
    x = BatchNormalization(name='bn10')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(name='pool4')(x)
    
    x = Conv2D(hp_filters * 8, (3, 3), padding='same', name='conv11')(x)
    x = BatchNormalization(name='bn11')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 8, (3, 3), padding='same', name='conv12')(x)
    x = BatchNormalization(name='bn12')(x)
    x = Activation('relu')(x)
    x = Conv2D(hp_filters * 8, (3, 3), padding='same', name='conv13')(x)
    x = BatchNormalization(name='bn13')(x)
    x = Activation('relu')(x)
    qwe = MaxPooling2D(name='pool5')(x)
    
    # Dense layers
    x = Flatten()(qwe)
    x = Dense(hp_filters * 16, activation='relu', name='fc1')(x)
    
    if hp.Boolean("dropouts"):
        x = Dropout(0.5)(x)
    if hp.Boolean("batch_normalization"):
        x = BatchNormalization()(x)
    
    x = Dense(hp_filters * 16, activation='relu', name='fc2')(x)
    
    if hp.Boolean("dropouts"):
        x = Dropout(0.5)(x)
    if hp.Boolean("batch_normalization"):
        x = BatchNormalization()(x)
    
    x = Dense(hp_filters * 16, activation='relu', name='fc3')(x)
    
    if hp.Boolean("4th_dense"):
        x = Dense(hp_filters * 16, activation='relu', name='fc4')(x)
        if hp.Boolean("dropouts"):
            x = Dropout(0.5)(x)
        if hp.Boolean("batch_normalization"):
            x = BatchNormalization()(x)
    
    # Reshape to reintroduce spatial dimensions
    print(qwe.shape)
    x = Reshape((qwe.shape[1], qwe.shape[2], hp_filters*4))(x)
    
    # Decoding path
    x = UpSampling2D()(x)
    x = Conv2DTranspose(hp_filters * 8, (3, 3), padding='same', name='deconv1')(x)
    x = BatchNormalization(name='bn14')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters * 8, (3, 3), padding='same', name='deconv2')(x)
    x = BatchNormalization(name='bn15')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters * 8, (3, 3), padding='same', name='deconv3')(x)
    x = BatchNormalization(name='bn16')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(hp_filters * 4, (3, 3), padding='same', name='deconv4')(x)
    x = BatchNormalization(name='bn17')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters * 4, (3, 3), padding='same', name='deconv5')(x)
    x = BatchNormalization(name='bn18')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters * 4, (3, 3), padding='same', name='deconv6')(x)
    x = BatchNormalization(name='bn19')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(hp_filters * 2, (3, 3), padding='same', name='deconv7')(x)
    x = BatchNormalization(name='bn20')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters * 2, (3, 3), padding='same', name='deconv8')(x)
    x = BatchNormalization(name='bn21')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters, (3, 3), padding='same', name='deconv9')(x)
    x = BatchNormalization(name='bn22')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(hp_filters, (3, 3), padding='same', name='deconv10')(x)
    x = BatchNormalization(name='bn23')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(hp_filters, (3, 3), padding='same', name='deconv11')(x)
    x = BatchNormalization(name='bn24')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv12')(x)
    x = BatchNormalization(name='bn25')(x)
    x = Activation('relu')(x)
    
    x = UpSampling2D()(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
    x = BatchNormalization(name='bn26')(x)
    
    # Output layer with sigmoid activation for binary segmentation
    outputs = Conv2D(2, (1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs);lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5]);model.compile(optimizer= tf.keras.optimizers.Adam(lr=lr), loss= ["binary_crossentropy"], metrics=['acc'])

    return model

In [46]:
tunerB = kt.BayesianOptimization(segnet,
                     objective='val_loss',
                     directory='my_dir',
                     max_trials= 10,
                     project_name='design_b',
                    #  seed=42,
                     )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)


tunerB.search(train_generator, epochs=100, validation_data=val_generator, callbacks=[stop_early, reduce_lr])

# Get the optimal hyperparameters
best_hpsB=tunerB.get_best_hyperparameters(num_trials=1)[0]



print(best_hpsB.values)

Trial 10 Complete [00h 01m 19s]
val_loss: 0.3817465901374817

Best val_loss So Far: 0.3729792535305023
Total elapsed time: 00h 25m 34s
{'filters': 16, 'dropouts': False, 'batch_normalization': True, '4th_dense': False, 'learning_rate': 0.001}


In [47]:
# best_hpsB.values['filters']=64

In [48]:
# Build the model with the best hp.
modelB = segnet(best_hpsB)
modelB.summary()

(None, 2, 2, 128)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 64, 64, 16)        448       
                                                                 
 bn1 (BatchNormalization)    (None, 64, 64, 16)        64        
                                                                 
 activation_25 (Activation)  (None, 64, 64, 16)        0         
                                                                 
 conv2 (Conv2D)              (None, 64, 64, 16)        2320      
                                                                 
 bn2 (BatchNormalization)    (None, 64, 64, 16)        64        
                                                                 
 activation_26 (Activation)  (None, 64, 6

In [49]:
# epochs = 512
# steps_per_epoch = 512 // batch_size + 1  # we usually consider 1 epoch to be
#                                             # the point where the model has seen
#                                             # all the training samples at least once
# min_lr = 1e-8
# factor = 0.1
# SCHEDULE_EPOCH = 200
# STEPS = 200


# historyB = {"history":{"loss":[],"mae":[],"val_loss":[],"val_mae":[]}}
# for e in range(epochs):
#     for i, (images, y_batch) in enumerate(train_generator):
#        new_y_batch = []
#        for x,img in enumerate(images):
#         array = np.ones(image_size+(3,))
#         array *= img>0
#         array[array>0] = y_batch[x]
#         new_y_batch.append(array)
#        new_y_batch = np.array(new_y_batch)
#        loss = modelB.train_on_batch(images, new_y_batch)  # train model for a single iteration
#     #    val = modelB.test_on_batch(images, new_y_batch)
#        if i >= steps_per_epoch:  # manually detect the end of the epoch
#             historyB["history"]["loss"].append(loss[0])
#             historyB["history"]["mae"].append(loss[1])
#             # print("EPOCH: {} LOSS: {:.6f} 2ND_METRIC: {:.6f}".format(e+1, loss[0], loss[1]))
#             break  
#     for i, (images, y_batch) in enumerate(val_generator):
#        new_y_batch = []
#        for x,img in enumerate(images):
#         array = np.ones(image_size+(3,))
#         array *= img>0
#         array[array>0] = y_batch[x]
#         new_y_batch.append(array)
#        new_y_batch = np.array(new_y_batch)
#     #    loss = modelB.train_on_batch(images, new_y_batch)  # train model for a single iteration
#        val = modelB.test_on_batch(images, new_y_batch)
#        if i >= steps_per_epoch:  # manually detect the end of the epoch
#             historyB["history"]["val_loss"].append(val[0])
#             historyB["history"]["val_mae"].append(val[1])
#             # print("EPOCH: {} LOSS: {:.6f} 2ND_METRIC: {:.6f}".format(e+1, loss[0], loss[1]))
#             curr_lr = modelB.optimizer.learning_rate
#             if e>=SCHEDULE_EPOCH and curr_lr>min_lr: 
#                K.set_value(modelB.optimizer.learning_rate, curr_lr*factor)
#                curr_lr = modelB.optimizer.learning_rate
#                SCHEDULE_EPOCH+=STEPS
#             # patience = 2
#             # if e>patience and curr_lr>min_lr:
#                # curr_loss = historyA["history"]["val_loss"][-1]
#                # for i in historyA["history"]["val_loss"][-patience-1:-1]:
#                #    if curr_loss >= i: patience-=1
#                #    if patience < 1:
#                      # K.set_value(modelA.optimizer.learning_rate, curr_lr*factor)
#                      # curr_lr = modelA.optimizer.learning_rate
#             break  
#     print("EPOCH: {} LOSS: {:.6f} MAE: {:.6f} VAL_LOSS: {:.6f} VAL_MAE: {:.6f}   CURR_LR {:.2E}".format(e+1, loss[0], loss[1],val[0],val[1], curr_lr.numpy()))
#     train_generator.on_epoch_end()  # this shuffles the data at the end of each epoch

In [50]:
# plt.figure(figsize=(10,4))

# plt.subplot(121)
# plt.plot(historyB["history"]["loss"][10:], color='g',alpha=.5)
# plt.plot(historyB["history"]["val_loss"][10:], color='r',alpha=.7)
# plt.legend(["Train","Val"])
# plt.ylabel("MSE")
# plt.xlabel("Epochs")

# plt.subplot(122)
# plt.plot(historyB["history"]["mae"][10:], color='g',alpha=.5)
# plt.plot(historyB["history"]["val_mae"][10:], color='r',alpha=.7)
# plt.legend(["Train","Val"])
# plt.ylabel("MAE")
# plt.xlabel("Epochs")

# plt.suptitle("Model B: SegNet")
# plt.show()

In [51]:
# np.array(historyB["history"]["loss"]).shape

In [52]:
# red_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                               patience=10, min_lr=1e-7)
# # stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# history = modelB.fit(train_generator, epochs=2,# validation_data=val_generator,
#                      callbacks=[red_lr])

In [53]:
# sample = next(train_generator)


In [54]:
# sample = next(train_generator)
# result = modelB(sample[0])
# for x,i in enumerate(result):
#     RGB  = cv2.cvtColor(sample[0][x],cv2.COLOR_HSV2RGB)
#     MASK = cv2.cvtColor(RGB, cv2.COLOR_RGB2GRAY) > 0
#     ARRAY = np.array(result[5]).reshape(image_size) * MASK
#     OUTPUT = np.array([x for x in ARRAY.flatten() if x > 0])
#     print(np.average(OUTPUT), np.average(sample[1][x]))

In [55]:

# plt.imshow(result[5])

In [56]:
# plt.imshow(sample[0][5])

In [57]:
# import cv2
# RGB  = cv2.cvtColor(sample[0][5],cv2.COLOR_HSV2RGB)
# MASK = cv2.cvtColor(RGB, cv2.COLOR_RGB2GRAY) > 0
# ARRAY = np.array(result[5]).reshape(image_size) * MASK


In [58]:
# AVE = np.average(ARRAY)
# TRUTH = sample[1][5]
# print(AVE)
# print(TRUTH)

In [59]:
# plt.imshow(ARRAY)
# plt.show()

In [60]:
# modelA.save('design_models/BEST/designA_V3.h5')
# modelB.save('design_models/BEST/designB_V3.h5')
# modelC.save('design_models/BEST/designC_V3.h5')

In [61]:
# from shutil import rmtree
# # removing directory 
# rmtree('my_dir') 

### Design C

In [149]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Conv2DTranspose, Dropout, Flatten, Dense, Reshape, AveragePooling2D, Concatenate
from tensorflow.keras import layers, initializers
import tensorflow as tf
import keras_tuner as kt

def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
    x = Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding=padding,
        use_bias=use_bias,
        kernel_initializer=initializers.HeNormal(),
    )(block_input)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

def DilatedSpatialPyramidPooling(dspp_input, num_filters):
    dims = dspp_input.shape
    x = AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, num_filters=num_filters, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, num_filters=num_filters,  kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, num_filters=num_filters,  kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, num_filters=num_filters,  kernel_size=3, dilation_rate=18)

    x = Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def DeeplabV3Plus(hp, image_size=(64, 64)):
    model_input = Input(shape=image_size+(3,))
    resnet50 = tf.keras.applications.ResNet50(
        weights='imagenet', include_top=False, input_tensor=model_input
    )
    for layer in resnet50.layers:
        layer.trainable=True
    
    hp_filters = hp.Choice('filters', values=[16, 32, 64])
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x, hp_filters*2)
    print(x.shape)
    x = Flatten()(x)
    x = Dense(hp_filters*hp_filters, activation='relu')(x)

    UNITS = hp_filters*hp_filters
    hp.Boolean("dropouts", default=False)
    hp.Boolean("batch_normalization", default=False)

    x = Dense(UNITS, activation='relu')(x)
    if hp.Boolean("dropouts"):
        x = Dropout(0.5)(x)
    if hp.Boolean("batch_normalization"):
        x = BatchNormalization()(x)
    x = Dense(UNITS, activation='relu')(x)
    if hp.Boolean("dropouts"):
        x = Dropout(0.5)(x)
    if hp.Boolean("batch_normalization"):
        x = BatchNormalization()(x)
    x = Dense(UNITS, activation='relu')(x)
    if hp.Boolean("4th_dense", default=False):
        x = Dense(UNITS, activation='relu')(x)
        if hp.Boolean("dropouts"):
            x = Dropout(0.5)(x)
        if hp.Boolean("batch_normalization"):
            x = BatchNormalization()(x)

    SIZE = image_size[0]
    temp = 0
    if hp_filters == 16:
        temp = hp_filters
    if hp_filters == 32:
        temp = hp_filters * 2
    if hp_filters == 64:
        temp = hp_filters * 4
    x = Reshape((SIZE // 16, SIZE // 16, temp))(x)

    x = UpSampling2D(size=(SIZE // 4 // x.shape[1], SIZE // 4 // x.shape[2]), interpolation="bilinear")(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=hp_filters, kernel_size=1)

    x = Concatenate(axis=-1)([x, input_b])
    x = convolution_block(x, num_filters=hp_filters*4)
    x = convolution_block(x, num_filters=hp_filters*4)
    x = UpSampling2D(size=(SIZE // x.shape[1], SIZE // x.shape[2]), interpolation="bilinear")(x)
    
    outputs = Conv2D(2, 1, activation='sigmoid')(x)
    
    model = Model(inputs=model_input, outputs=outputs)
    lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr), loss=["binary_crossentropy"], metrics=['acc'])

    return model


In [150]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, applications

# def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
#     x = layers.Conv2D(
#         num_filters,
#         kernel_size=kernel_size,
#         dilation_rate=dilation_rate,
#         padding=padding,
#         use_bias=use_bias,
#         kernel_initializer=tf.keras.initializers.HeNormal(),
#     )(block_input)
#     x = layers.BatchNormalization()(x)
#     return tf.nn.relu(x)

# def DilatedSpatialPyramidPooling(dspp_input):
#     dims = dspp_input.shape
#     x = layers.AveragePooling2D(pool_size=(dims[1], dims[2]))(dspp_input)
#     x = convolution_block(x, num_filters=256, kernel_size=1, use_bias=True)
#     out_pool = layers.UpSampling2D(
#         size=(dims[1] // x.shape[1], dims[2] // x.shape[2]), interpolation="bilinear",
#     )(x)

#     out_1 = convolution_block(dspp_input, num_filters=256, kernel_size=1, dilation_rate=1)
#     out_6 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=6)
#     out_12 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=12)
#     out_18 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=18)

#     x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
#     output = convolution_block(x, num_filters=256, kernel_size=1)
#     return output

# def DeeplabV3Plus(hp, image_size=(64, 64, 3), num_classes=2):
#     base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=image_size)
#     base_model_output = base_model.get_layer("conv4_block6_2_relu").output

#     x = DilatedSpatialPyramidPooling(base_model_output)

#     input_a = layers.UpSampling2D(size=(image_size[0] // 4 // x.shape[1], image_size[1] // 4 // x.shape[2]), interpolation="bilinear")(x)
#     input_b = base_model.get_layer("conv2_block3_2_relu").output
#     input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

#     x = layers.Concatenate(axis=-1)([input_a, input_b])
#     x = convolution_block(x, num_filters=256)
#     x = convolution_block(x, num_filters=256)
#     x = layers.UpSampling2D(size=(image_size[0] // x.shape[1], image_size[1] // x.shape[2]), interpolation="bilinear")(x)
#     x = layers.Conv2D(2, (1, 1), padding="same")(x)
#     outputs = tf.keras.layers.Activation('sigmoid')(x)

#     model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
#     lr = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
#     model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr), loss=["binary_crossentropy"], metrics=['acc'])
#     return model


In [151]:
tunerC = kt.BayesianOptimization(DeeplabV3Plus,
                     objective='val_loss',
                     directory='my_dir',
                     max_trials= 10,
                     project_name='design_c',
                    #  seed=42,
                     )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

tunerC.search(train_generator, epochs=100, validation_data=val_generator, callbacks=[stop_early, reduce_lr])

# Get the optimal hyperparameters
best_hpsC=tunerC.get_best_hyperparameters(num_trials=1)[0]

print(best_hpsC.values)

Trial 10 Complete [00h 02m 19s]
val_loss: 0.3647870719432831

Best val_loss So Far: 0.3272910416126251
Total elapsed time: 00h 34m 56s
{'filters': 64, 'dropouts': True, 'batch_normalization': False, '4th_dense': False, 'learning_rate': 0.0001}


In [152]:
# best_hpsC.values['filters']=64
# best_hpsC.values['learning_rate']=1e-6
# Build the model with the best hp.
modelC = DeeplabV3Plus(best_hpsC)
modelC.summary()



(None, 4, 4, 256)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                          

In [153]:
# epochs = 512
# steps_per_epoch = 512 // batch_size + 1  # we usually consider 1 epoch to be
#                                             # the point where the model has seen
#                                             # all the training samples at least once
# min_lr = 1e-8
# factor = 0.1
# SCHEDULE_EPOCH = 200
# STEPS = 200

# historyC = {"history":{"loss":[],"mae":[],"val_loss":[],"val_mae":[]}}
# for e in range(epochs):
#     for i, (images, y_batch) in enumerate(train_generator):
#        new_y_batch = []
#        for x,img in enumerate(images):
#         array = np.ones(image_size+(3,))
#         array *= img>0
#         array[array>0] = y_batch[x]
#         new_y_batch.append(array)
#        new_y_batch = np.array(new_y_batch)
#        loss = modelC.train_on_batch(images, new_y_batch)  # train model for a single iteration
#     #    val = modelB.test_on_batch(images, new_y_batch)
#        if i >= steps_per_epoch:  # manually detect the end of the epoch
#             historyC["history"]["loss"].append(loss[0])
#             historyC["history"]["mae"].append(loss[1])
#             # print("EPOCH: {} LOSS: {:.6f} 2ND_METRIC: {:.6f}".format(e+1, loss[0], loss[1]))
#             break  
#     for i, (images, y_batch) in enumerate(val_generator):
#        new_y_batch = []
#        for x,img in enumerate(images):
#         array = np.ones(image_size+(3,))
#         array *= img>0
#         array[array>0] = y_batch[x]
#         new_y_batch.append(array)
#        new_y_batch = np.array(new_y_batch)
#     #    loss = modelB.train_on_batch(images, new_y_batch)  # train model for a single iteration
#        val = modelC.test_on_batch(images, new_y_batch)
#        if i >= steps_per_epoch:  # manually detect the end of the epoch
#             historyC["history"]["val_loss"].append(val[0])
#             historyC["history"]["val_mae"].append(val[1])
#             # print("EPOCH: {} LOSS: {:.6f} 2ND_METRIC: {:.6f}".format(e+1, loss[0], loss[1]))
#             curr_lr = modelC.optimizer.learning_rate
#             if e>=SCHEDULE_EPOCH and curr_lr>min_lr: 
#                K.set_value(modelC.optimizer.learning_rate, curr_lr*factor)
#                curr_lr = modelC.optimizer.learning_rate
#                SCHEDULE_EPOCH+=STEPS
#             # patience = 2
#             # if e>patience and curr_lr>min_lr:
#                # curr_loss = historyA["history"]["val_loss"][-1]
#                # for i in historyA["history"]["val_loss"][-patience-1:-1]:
#                #    if curr_loss >= i: patience-=1
#                #    if patience < 1:
#                      # K.set_value(modelA.optimizer.learning_rate, curr_lr*factor)
#                      # curr_lr = modelA.optimizer.learning_rate
#             break  
#     print("EPOCH: {} LOSS: {:.6f} MAE: {:.6f} VAL_LOSS: {:.6f} VAL_MAE: {:.6f}   CURR_LR {:.2E}".format(e+1, loss[0], loss[1],val[0],val[1], curr_lr.numpy()))
#     train_generator.on_epoch_end()  # this shuffles the data at the end of each epoch

In [154]:
# plt.figure(figsize=(10,4))

# plt.subplot(121)
# plt.plot(historyC["history"]["loss"][10:], color='g',alpha=.5)
# plt.plot(historyC["history"]["val_loss"][10:], color='r',alpha=.7)
# plt.legend(["Train","Val"])
# plt.ylabel("MSE")
# plt.xlabel("Epochs")

# plt.subplot(122)
# plt.plot(historyC["history"]["mae"][10:], color='g',alpha=.5)
# plt.plot(historyC["history"]["val_mae"][10:], color='r',alpha=.7)
# plt.legend(["Train","Val"])
# plt.ylabel("MAE")
# plt.xlabel("Epochs")

# plt.suptitle("Model C: DeepLabV3+")
# plt.show()

In [155]:
# sample = next(train_generator)
# result = modelC(sample[0])
# for x,i in enumerate(result):
#     RGB  = cv2.cvtColor(sample[0][x],cv2.COLOR_HSV2RGB)
#     MASK = cv2.cvtColor(RGB, cv2.COLOR_RGB2GRAY) > 0
#     ARRAY = np.array(result[5]).reshape(image_size) * MASK
#     OUTPUT = np.array([x for x in ARRAY.flatten() if x > 0])
#     print(np.average(OUTPUT), np.average(sample[1][x]))

In [156]:
# modelC.save('design_models/designC.h5')

In [157]:
# from shutil import rmtree
# # removing directory 
# rmtree('my_dir') 

## Evaluating Constraints

### MANUFACTURABILITY: TRAINING TIME

In [158]:
epochs = 128
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.1,
                              patience=6, min_lr=1e-9)
es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=16, mode="min", restore_best_weights=True)

In [159]:
## DESIGN A
import time 
start = time.time()
historyA = modelA.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=2, callbacks = [es, reduce_lr])
stop = time.time()
print(f"Training time: {stop - start}s")

# ## FUNCTIONALITY: INFERENCE TIME
# modelC.evaluate(train_generator[1][0][0].reshape(1,32,32,3))

Epoch 1/128
79/79 - 4s - loss: 0.3924 - acc: 0.8617 - val_loss: 0.4071 - val_acc: 0.8421 - lr: 0.0010 - 4s/epoch - 48ms/step
Epoch 2/128
79/79 - 2s - loss: 0.3904 - acc: 0.8605 - val_loss: 0.4206 - val_acc: 0.8302 - lr: 0.0010 - 2s/epoch - 30ms/step
Epoch 3/128
79/79 - 2s - loss: 0.3908 - acc: 0.8638 - val_loss: 0.3989 - val_acc: 0.8510 - lr: 0.0010 - 2s/epoch - 28ms/step
Epoch 4/128
79/79 - 2s - loss: 0.3889 - acc: 0.8650 - val_loss: 0.3902 - val_acc: 0.8020 - lr: 0.0010 - 2s/epoch - 29ms/step
Epoch 5/128
79/79 - 2s - loss: 0.3872 - acc: 0.8643 - val_loss: 0.3781 - val_acc: 0.8492 - lr: 0.0010 - 2s/epoch - 29ms/step
Epoch 6/128
79/79 - 2s - loss: 0.3886 - acc: 0.8632 - val_loss: 0.3865 - val_acc: 0.8074 - lr: 0.0010 - 2s/epoch - 29ms/step
Epoch 7/128
79/79 - 2s - loss: 0.3876 - acc: 0.8629 - val_loss: 0.3777 - val_acc: 0.8475 - lr: 0.0010 - 2s/epoch - 29ms/step
Epoch 8/128
79/79 - 2s - loss: 0.3853 - acc: 0.8658 - val_loss: 0.3681 - val_acc: 0.8515 - lr: 0.0010 - 2s/epoch - 28ms/step


In [160]:
## DESIGN B
import time 
start = time.time()
historyB = modelB.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=2, callbacks = [es, reduce_lr])
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/128
79/79 - 6s - loss: 0.6807 - acc: 0.3529 - val_loss: 0.6502 - val_acc: 0.3232 - lr: 0.0010 - 6s/epoch - 73ms/step
Epoch 2/128
79/79 - 2s - loss: 0.6280 - acc: 0.3527 - val_loss: 0.6214 - val_acc: 0.4018 - lr: 0.0010 - 2s/epoch - 25ms/step
Epoch 3/128
79/79 - 2s - loss: 0.5926 - acc: 0.3451 - val_loss: 0.5827 - val_acc: 0.3339 - lr: 0.0010 - 2s/epoch - 25ms/step
Epoch 4/128
79/79 - 2s - loss: 0.5620 - acc: 0.3403 - val_loss: 0.5623 - val_acc: 0.3590 - lr: 0.0010 - 2s/epoch - 25ms/step
Epoch 5/128
79/79 - 2s - loss: 0.5329 - acc: 0.3416 - val_loss: 0.5597 - val_acc: 0.4265 - lr: 0.0010 - 2s/epoch - 25ms/step
Epoch 6/128
79/79 - 2s - loss: 0.5056 - acc: 0.3363 - val_loss: 0.5129 - val_acc: 0.3662 - lr: 0.0010 - 2s/epoch - 25ms/step
Epoch 7/128
79/79 - 2s - loss: 0.4818 - acc: 0.3315 - val_loss: 0.4766 - val_acc: 0.3231 - lr: 0.0010 - 2s/epoch - 25ms/step
Epoch 8/128
79/79 - 2s - loss: 0.4633 - acc: 0.3314 - val_loss: 0.4718 - val_acc: 0.3539 - lr: 0.0010 - 2s/epoch - 25ms/step


In [161]:
## DESIGN C
import time 
start = time.time()
historyC = modelC.fit(train_generator, epochs=epochs, validation_data=val_generator, verbose=2, callbacks = [es, reduce_lr])
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/128
79/79 - 10s - loss: 0.4586 - acc: 0.7431 - val_loss: 2.2830 - val_acc: 0.6330 - lr: 1.0000e-04 - 10s/epoch - 122ms/step
Epoch 2/128
79/79 - 4s - loss: 0.3839 - acc: 0.7005 - val_loss: 5.7365 - val_acc: 0.7122 - lr: 1.0000e-04 - 4s/epoch - 51ms/step
Epoch 3/128
79/79 - 4s - loss: 0.3749 - acc: 0.7309 - val_loss: 9.0444 - val_acc: 0.7354 - lr: 1.0000e-04 - 4s/epoch - 53ms/step
Epoch 4/128
79/79 - 4s - loss: 0.3721 - acc: 0.7393 - val_loss: 6.4444 - val_acc: 0.7225 - lr: 1.0000e-04 - 4s/epoch - 51ms/step
Epoch 5/128
79/79 - 4s - loss: 0.3681 - acc: 0.7460 - val_loss: 4.1656 - val_acc: 0.8519 - lr: 1.0000e-04 - 4s/epoch - 52ms/step
Epoch 6/128
79/79 - 4s - loss: 0.3669 - acc: 0.7464 - val_loss: 1.6282 - val_acc: 0.6987 - lr: 1.0000e-04 - 4s/epoch - 53ms/step
Epoch 7/128
79/79 - 4s - loss: 0.3652 - acc: 0.7405 - val_loss: 1.1341 - val_acc: 0.7152 - lr: 1.0000e-04 - 4s/epoch - 51ms/step
Epoch 8/128
79/79 - 4s - loss: 0.3655 - acc: 0.7731 - val_loss: 0.5423 - val_acc: 0.7761 - lr:

### ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND

In [162]:
## Design A
## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND
# Calculae FLOPS
from keras_flops import get_flops
flopsA = get_flops(modelA, batch_size=1)
print(f"FLOPS: {flopsA}")

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
FLOPS: 494470548


In [163]:
## Design B
## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND
# Calculae FLOPS
from keras_flops import get_flops
flopsB = get_flops(modelB, batch_size=1)
print(f"FLOPS: {flopsB}")

FLOPS: 217993206


In [164]:
## Design C
## ECONOMIC: FLOATING-POINT OPERATIONS PER SECOND
# Calculae FLOPS
from keras_flops import get_flops
flopsC = get_flops(modelC, batch_size=1)
print(f"FLOPS: {flopsC}")

FLOPS: 1629751046


### FUNCTIONALITY: INFERENCE TIME

In [166]:
inference_sample = train_generator[1][0][0].reshape(1,64,64,3)

In [167]:
## Design A
## FUNCTIONALITY: INFERENCE TIME
start = time.time()
modelA.predict(inference_sample)
stop = time.time()
print(f"Inference time: {(stop - start)*1e3:.4f}ms")


1/1 [==============================] - 1s 739ms/step
Inference time: 1001.2958ms


In [168]:
## Design B
## FUNCTIONALITY: INFERENCE TIME
start = time.time()
modelB.predict(inference_sample)
stop = time.time()
print(f"Inference time: {(stop - start)*1e3:.4f}ms")


1/1 [==============================] - 1s 537ms/step
Inference time: 568.9769ms


In [169]:
## Design AC
## FUNCTIONALITY: INFERENCE TIME
start = time.time()
modelC.predict(inference_sample)
stop = time.time()
print(f"Inference time: {(stop - start)*1e3:.4f}ms")


1/1 [==============================] - 1s 900ms/step
Inference time: 956.5411ms


### PERFORMANCE: COEFFICIENT OF DETERMINATION

In [170]:
## Prepare set of x values and y values for performance constraint
X_values,y_values = [],[]
for i in range(100):
    values = next(val_generator)
    # for j in range(values[0].shape[0]):
    X_values.append(values[0])
    y_values.append(values[1])

## create X_values generator
gen_X_values_1 = (x for x in X_values)
gen_X_values_2 = (x for x in X_values)
gen_X_values_3 = (x for x in X_values)
y_values = [y  for y_set in y_values for y in y_set]
y_values_0 = np.array(y_values)[:,:,:,0]
y_values_1 = np.array(y_values)[:,:,:,1]

In [171]:
import numpy as np
from sklearn.metrics import r2_score

y_values = np.array(y_values)
predictionsA = modelA.predict(gen_X_values_1)

y_values_flat_0 = y_values[:, :, :, 0].flatten()
predictionsA_flat_0 = predictionsA[:, :, :, 0].flatten()

y_values_flat_1 = y_values[:, :, :, 1].flatten()
predictionsA_flat_1 = predictionsA[:, :, :, 1].flatten()

# Calculate R^2 score for each class
r2_score_0 = r2_score(y_values_flat_0, predictionsA_flat_0)
r2_score_1 = r2_score(y_values_flat_1, predictionsA_flat_1)

print(f'R^2 for class 0: {r2_score_0:.4f}')
print(f'R^2 for class 1: {r2_score_1:.4f}')

print(f"Average R^2 score: {(r2_score_0+r2_score_1)/2:.4f}")


100/100 [==============================] - 3s 10ms/step
R^2 for class 0: 0.6145
R^2 for class 1: 0.4850
Average R^2 score: 0.5497


In [172]:
## Design B
## PERFORMANCE: COEFFICIENT OF DETERMINATION
from sklearn.metrics import roc_auc_score
predictionsB = modelB.predict(gen_X_values_2)

y_values_flat_0 = y_values[:, :, :, 0].flatten()
predictionsB_flat_0 = predictionsB[:, :, :, 0].flatten()

y_values_flat_1 = y_values[:, :, :, 1].flatten()
predictionsB_flat_1 = predictionsB[:, :, :, 1].flatten()

# Calculate R^2 score for each class
r2_score_0 = r2_score(y_values_flat_0, predictionsB_flat_0)
r2_score_1 = r2_score(y_values_flat_1, predictionsB_flat_1)

print(f'R^2 for class 0: {r2_score_0:.4f}')
print(f'R^2 for class 1: {r2_score_1:.4f}')

print(f"Average R^2 score: {(r2_score_0+r2_score_1)/2:.4f}")



100/100 [==============================] - 1s 6ms/step
R^2 for class 0: 0.5996
R^2 for class 1: 0.2402
Average R^2 score: 0.4199


In [173]:
## Design B
## PERFORMANCE: COEFFICIENT OF DETERMINATION
from sklearn.metrics import roc_auc_score
predictionsC = modelC.predict(gen_X_values_3)

y_values_flat_0 = y_values[:, :, :, 0].flatten()
predictionsC_flat_0 = predictionsC[:, :, :, 0].flatten()

y_values_flat_1 = y_values[:, :, :, 1].flatten()
predictionsC_flat_1 = predictionsC[:, :, :, 1].flatten()

# Calculate R^2 score for each class
r2_score_0 = r2_score(y_values_flat_0, predictionsC_flat_0)
r2_score_1 = r2_score(y_values_flat_1, predictionsC_flat_1)

print(f'R^2 for class 0: {r2_score_0:.4f}')
print(f'R^2 for class 1: {r2_score_1:.4f}')

print(f"Average R^2 score: {(r2_score_0+r2_score_1)/2:.4f}")



100/100 [==============================] - 2s 12ms/step
R^2 for class 0: -0.5446
R^2 for class 1: -0.4950
Average R^2 score: -0.5198


### EFFICIENCY: STORAGE CONSUMPTION

In [174]:
## EFFICIENCY: STORAGE CONSUMPTION
weightsA = modelA.get_weights()
total_sizeA = 0
for weight in weightsA:
    total_sizeA += tf.size(weight).numpy()

print(f"Total model weight size in megabytes: {total_sizeA*8e-6:.4f}")

Total model weight size in megabytes: 99.6718


In [175]:
## EFFICIENCY: STORAGE CONSUMPTION
weightsB = modelB.get_weights()
total_sizeB = 0
for weight in weightsB:
    total_sizeB += tf.size(weight).numpy()

print(f"Total model weight size in megabytes: {total_sizeB*8e-6:.4f}")

Total model weight size in megabytes: 13.9723


In [176]:
## EFFICIENCY: STORAGE CONSUMPTION
weightsC = modelC.get_weights()
total_sizeC = 0
for weight in weightsC:
    total_sizeC += tf.size(weight).numpy()

print(f"Total model weight size in megabytes: {total_sizeC*8e-6:.4f}")

Total model weight size in megabytes: 623.7215


## Saving final trained and constrained models

In [177]:
modelA.save('design_models/designA_v4.h5')
modelB.save('design_models/designB_v4.h5')
modelC.save('design_models/designC_v4.h5')